Init Container
==============

Init Container sind Spezialcontainer, die vor App-Containern in einem Pod ausgeführt werden.

Das nachfolgende Beispiel definiert einen einfachen Pod mit zwei Init-Containern. 

Der erste wartet auf `myservice` und der zweite wartet auf `mydb`. Sobald beide Init-Container abgeschlossen sind, führt der Pod den App-Container in seinem `spec` Abschnitt aus.

In [ ]:
%%bash
cat <<%EOF% | kubectl apply -f -
apiVersion: v1
kind: Pod
metadata:
  name: myapp-pod
  labels:
    app: myapp
spec:
  containers:
  - name: myapp-container
    image: busybox:1.28
    command: ['sh', '-c', 'echo The app is running! && sleep 3600']
  initContainers:
  - name: init-myservice
    image: busybox:1.28
    command: ['sh', '-c', 'until nslookup myservice; do echo waiting for myservice; sleep 2; done;']
  - name: init-mydb
    image: busybox:1.28
    command: ['sh', '-c', 'until nslookup mydb; do echo waiting for mydb; sleep 2; done;']
%EOF%

Anzeige des Status

In [ ]:
! kubectl get pods/myapp-pod
! kubectl describe pods/myapp-pod

Durch Erstellung des Services `myservices` wird der erste Init Container ausgeführt

In [ ]:
%%bash
cat <<%EOF% | kubectl apply -f -
apiVersion: v1
kind: Service
metadata:
  name: myservice
spec:
  ports:
  - protocol: TCP
    port: 80
    targetPort: 9376
%EOF%

In [ ]:
! kubectl get pods/myapp-pod

Und wenn wir den zweiten Services einrichten, wird der zweite Init Container und anschliessen My-App gestartet.

In [ ]:
%%bash
cat <<%EOF% | kubectl apply -f -
apiVersion: v1
kind: Service
metadata:
  name: mydb
spec:
  ports:
  - protocol: TCP
    port: 80
    targetPort: 9377
%EOF%

In [ ]:
! kubectl get pods/myapp-pod
! kubectl logs pods/myapp-pod